# 신경망을 이용한 예측 : 회귀문제

### 주택 가격 예측

- 1970년 중반 보스턴 주택 가격 데이터셋
- 데이터 506개: 훈련 샘플-404개, 테스트 샘플-102개
- 14개 변수
  - CRIM : 자치시(town) 별 1인당 범죄율
  - ZN : 25,000 평방피트를 초과하는 거주지역의 비율
  - INDUS : 비소매상업지역이 점유하고 있는 토지의 비율
  - CHAS : 찰스강에 대한 더미변수(강의 경계에 위치한 경우는 1, 아니면 0)
  - NOX : 10ppm 당 농축 일산화질소
  - RM :  주택 1가구당 평균 방의 개수
  - AGE : 1940년 이전에 건축된 소유주택의 비율
  - DIS : 5개의 보스턴 직업센터까지의 접근성 지수
  - RAD : 방사형 도로까지의 접근성 지수
  - TAX : 10,000 달러 당 재산세율
  - PTRATIO : 자치시(town)별 학생/교사 비율
  - B : 1000(Bk-0.63)^2, 여기서 Bk는 자치시별 흑인의 비율을 말함
  - LSTAT : 모집단의 하위계층의 비율(%)
  - MEDV : 본인 소유의 주택가격(중앙값) (단위: $1,000)

### 데이터 로드

In [1]:
from tensorflow.keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [2]:
train_data.shape

(404, 13)

In [3]:
test_data.shape

(102, 13)

In [4]:
train_data

array([[1.23247e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.96900e+02, 1.87200e+01],
       [2.17700e-02, 8.25000e+01, 2.03000e+00, ..., 1.47000e+01,
        3.95380e+02, 3.11000e+00],
       [4.89822e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.75520e+02, 3.26000e+00],
       ...,
       [3.46600e-02, 3.50000e+01, 6.06000e+00, ..., 1.69000e+01,
        3.62250e+02, 7.83000e+00],
       [2.14918e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
        2.61950e+02, 1.57900e+01],
       [1.43900e-02, 6.00000e+01, 2.93000e+00, ..., 1.56000e+01,
        3.76700e+02, 4.38000e+00]])

### 데이터 준비

- 데이터 정규화
  - 상이한 스케일을 가진 값을 신경망에 주입하면 문제가 발생
  - 입력 데이터에 있는 각 특성을 평균은 0, 표준편차 1이 되도록 정규화함

In [5]:
train_data.mean(axis=0)

array([3.74511057e+00, 1.14801980e+01, 1.11044307e+01, 6.18811881e-02,
       5.57355941e-01, 6.26708168e+00, 6.90106436e+01, 3.74027079e+00,
       9.44059406e+00, 4.05898515e+02, 1.84759901e+01, 3.54783168e+02,
       1.27408168e+01])

In [6]:
train_data.std(axis=0)

array([9.22929073e+00, 2.37382770e+01, 6.80287253e+00, 2.40939633e-01,
       1.17147847e-01, 7.08908627e-01, 2.79060634e+01, 2.02770050e+00,
       8.68758849e+00, 1.66168506e+02, 2.19765689e+00, 9.39946015e+01,
       7.24556085e+00])

In [7]:
# 데이터 정규화

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

In [8]:
train_data.mean(axis=0)

array([-1.01541438e-16,  1.09923072e-17,  1.74337992e-15, -1.26686340e-16,
       -5.25377321e-15,  6.41414864e-15,  2.98441140e-16,  4.94653823e-16,
        1.12671149e-17, -1.98136337e-16,  2.36686358e-14,  5.95679996e-15,
        6.13920356e-16])

In [9]:
train_data.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

### 1) 신경망 모델 정의 및 컴파일

- 샘플 개수가 적으므로 64개 유닛을 가진 2개의 은닉층으로 네트워크 구성


- 모델 정의 및 컴파일을 위한 함수 생성 build_model()

In [13]:
from tensorflow.keras import models
from tensorflow.keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    # loss에서 이진 분류일 때는 birnary cross entropy
    # loss에서 다중 분류일 때는 categorical cross entropy
    
    return model

### 2) 모델 훈련 및 검증

**K-fold cross-validation을 사용**

In [14]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('처리중인 폴드 #', i)
    
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples:(i+1)*num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate([train_data[:i*num_val_samples],
                                           train_data[(i+1)*num_val_samples:]],
                                           axis=0)
    partial_train_targets = np.concatenate([train_targets[:i*num_val_samples], 
                                              train_targets[(i+1)*num_val_samples:]],
                                             axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    model.fit(partial_train_data, partial_train_targets,
                epochs=num_epochs, batch_size=1, verbose=0)

    # 검증 세트로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


In [15]:
all_scores

[2.2089858055114746,
 2.4882256984710693,
 2.6164145469665527,
 2.4481425285339355]

In [16]:
np.mean(all_scores)

2.440442144870758

- 각 폴드에서 검증 점수를 로그에 저장하기

  - epochs를 500으로 하여 다시 훈련하여 검증

In [17]:
num_epochs = 500
all_mae_histories = []

for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples:(i+1)*num_val_samples]

    partial_train_data = np.concatenate([train_data[:i*num_val_samples],
                                       train_data[(i+1)*num_val_samples:]],
                                       axis=0)
    partial_train_targets = np.concatenate([train_targets[:i*num_val_samples], 
                                          train_targets[(i+1)*num_val_samples:]],
                                         axis=0)
  
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                      validation_data=(val_data, val_targets),
                      epochs=num_epochs, batch_size=1, verbose=0)
  
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

처리중인 폴드 # 0


KeyboardInterrupt: 

In [ ]:
avg_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
avg_mae_hist

In [ ]:
np.argmin(avg_mae_hist)

In [ ]:
np.argmax(avg_mae_hist)

### 3) 모델 성능 시각화

- 모든 폴드에 대해 에포크별 MAE 평균점수

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 6))
plt.plot(range(1, len(average_mae_history)+1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.grid()

plt.show()

- 처음 10개 데이터 포인트를 제외한 검증 점수 그래프
  - 부드러운 곡선을 얻기 위해 각 포인트를 이전 포인트의 지수이동평균으로 대체

In [ ]:
def smooth_curve(points, weight=0.9):
    smoothed_points = []
    for point in points:
    if smoothed_points:
        previous = smoothed_points[-1]
        smoothed_points.append(previous * weight + point*(1-weight))
    else:
        smoothed_points.append(point)
    return smoothed_points

In [ ]:
smooth_mae_history = smooth_curve(average_mae_history[10:])

In [ ]:
np.argmin(smooth_mae_history)

In [ ]:
plt.plot(range(1, len(smooth_mae_history)+1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

### 4) 최종 모델 훈련하기

- 에포크수, 은닉층의 크기 등 모델의 여러 매개변수에 대한 튜닝이 끝나면 모든 훈련데이터를 사용하여 모델을 훈련함

In [ ]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)

### 5) 테스트 데이터로 성능 확인

In [ ]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score

## [정리]

- 회귀는 분류에서 사용했던 것과 다른 손실함수 사용 : 평균제곱오차(MSE)
- 회귀에서 사용되는 평가지표 : 평균절대오차(MAE)
- 입력 데이터의 특성이 서로 다른 범위를 가지면 전처리 단계에서 각 특성을 개별적으로 스케일링해야 함 -> 정규화
- 가용한 데이터가 적다면 K-fold 교차검증을 사용하는 것이 신뢰할 수 있는 모델 평가방법임
- 가용한 훈련 데이터가 적다면 과대적합을 피하기 위해 은닉층의 수를 줄임 모델이 좋음(일반적으로 1개 또는 2개)